<a href="https://colab.research.google.com/github/MrymHkmbdi/GenreDetector/blob/main/iml_GenreDetector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import re
import json
import numpy as np
import pandas as pd
from nltk.stem import PorterStemmer
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize

In [ ]:
pip install clean-text

In [ ]:
from cleantext import clean

In [ ]:
train_set = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/GenreDetector/train.csv', encoding='Windows-1252')
test_set = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/GenreDetector/test.csv', encoding='Windows-1252')

In [ ]:
train_set.head()

In [ ]:
all_genres = []
for i in range(1,len(test_set)):
    temp = test_set.loc[i,"genres"].replace("\'", "\"")
    temp= json.loads(temp)
    a=[d['name'] for d in temp]
    for a1 in a:
        if( not (a1 in all_genres)):
            all_genres.append(a1)
print(all_genres)

In [ ]:
df_lables=[]
column_names=[]
for i  in range(0,20):
    column_names.append(str(i))

df_lables = pd.DataFrame(columns = column_names)

In [ ]:
df_lables.head()

In [ ]:
df_lables.to_csv("/content/drive/MyDrive/Colab Notebooks/GenreDetector/test_lables.csv")

In [ ]:
df_lables.to_csv("/content/drive/MyDrive/Colab Notebooks/GenreDetector/train_lables.csv")

In [ ]:
genres = ['Action', 'Adventure', 'Horror', 'Thriller', 'Science Fiction', 'Drama', 'Family', 'Animation', 'History', 'Romance', 'Crime', 'Comedy', 'Music', 'War', 'Fantasy', 'Mystery', 'Documentary', 'Foreign', 'Western']

In [ ]:
train_set=train_set.fillna(0)
test_set=test_set.fillna(0)

In [ ]:
cleans_data=[]
df_vectors= []
ps = PorterStemmer()
for i in range(0,len(test_set)):
    vector=[0]*20    
    temp = test_set.loc[i,"genres"].replace("\'", "\"")
    temp= json.loads(temp)
    a=[d['name'] for d in temp]
    for a1 in a:
        j=genres.index(a1)
        df_lables.loc[i,str(j)]=1
        vector[j]=1
    df_vectors.append(vector)
    clean_data=clean(test_set.loc[i,"overview"],
        fix_unicode=True,               # fix various unicode errors
        to_ascii=True,                  # transliterate to closest ASCII representation
        lower=True,                     # lowercase text
        no_line_breaks=True,           # fully strip line breaks as opposed to only normalizing them
        no_urls=True,                  # replace all URLs with a special token
        no_emails=True,                # replace all email addresses with a special token
        no_phone_numbers=True,         # replace all phone numbers with a special token
        no_numbers=False,               # replace all numbers with a special token
        no_digits=False,                # replace all digits with a special token
        no_currency_symbols=True,      # replace all currency symbols with a special token
        no_punct=True,                 # remove punctuations
        replace_with_punct="",          # instead of removing punctuations you may replace them
        replace_with_url="<URL>",
        replace_with_email="<EMAIL>",
        replace_with_phone_number="<PHONE>",
        replace_with_number="<NUMBER>",
        replace_with_digit="0",
        replace_with_currency_symbol="<CUR>",
        lang="en"                       # set to 'de' for German special handling
    )
    clean_data_col=""
    words=re.sub("[^\w]", " ", clean_data).split()
    for w in words:
        clean_data_col = clean_data_col + " "+ ps.stem(w)
    test_set.loc[i,"clean_data"]=clean_data_col
    cleans_data.append(clean_data_col)
test_set["clean"]=cleans_data
test_set.to_csv("/content/drive/MyDrive/Colab Notebooks/GenreDetector/cleaned_test.csv")

In [ ]:
def get_mean_vector(word2vec_model, words):
    # remove out-of-vocabulary words
    words=re.sub("[^\w]", " ",  words).split()
    sent_vec = np.zeros(25) # as word vectors are of zero length 50, 
    #you might need to change this to 300 if you use google's w2v
    cnt_words =0; # num of words with a valid vector in the sentence/review
    
    for word in words: # for each word in a review/sentence
        if word in word_vectors:
            vec = wv[word]
            sent_vec += vec
            cnt_words += 1
    if cnt_words != 0:
        sent_vec /= cnt_words
    return sent_vec

In [ ]:
import gensim.downloader
from gensim.test.utils import common_texts

from gensim.models import Word2Vec

model = Word2Vec(sentences=common_texts,window=5, min_count=1, workers=4)

In [ ]:
glove_vectors = gensim.downloader.load('glove-twitter-25')

[==================================================] 100.0% 104.8/104.8MB downloaded


In [ ]:
from gensim.models import KeyedVectors

# Store just the words + their trained embeddings.

word_vectors = glove_vectors.wv

word_vectors.save("word2vec.wordvectors")

# Load back with memory-mapping = read-only, shared across processes.

wv = KeyedVectors.load("word2vec.wordvectors", mmap='r')


In [ ]:
meanvectors=[]
for i in range(0,len(train_set)):
    mean_vector=get_mean_vector(wv,str(train_set.loc[i,"overview"]))
    meanvectors.append(mean_vector)
meanvectors[0]

In [ ]:
train_set["word2vec"]=meanvectors

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier

In [ ]:
meanvectors_test=[]
for i in range(0,len(test_set)):
    mean_vector=get_mean_vector(wv,str(test_set.loc[i,"overview"]))
    meanvectors_test.append(mean_vector)

In [ ]:
test_set["word2vec"]=meanvectors_test